In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [7]:
# deleting a column, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
df = pd.read_csv(filepath_or_buffer='../data/rewatches.csv',
                         sep=",",
                         skiprows=1,
                         header=None,
                         dtype={0: int, 1: int,2: int},
                         engine='python')

Reader.df_col_normalize(df,2)
matrix_df=df.sort_values(by=[0,1])
matrix_df.to_csv('../output/out_2.csv', index=False)

In [1]:
# normalizing a column, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
df = pd.read_csv(filepath_or_buffer='../data/rewatches.csv',
                 sep=",",
                 skiprows=1,
                 header=None,
                 dtype={0: int, 1: int,2: int},
                 engine='python')

df[2]= Reader.df_col_normalize(df,2)
df=df.sort_values(by=[0,1])
df.to_csv('../output/out_2.csv', index=False)

In [6]:
# normalizing a column with substitution, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
matrix_df = pd.read_csv(filepath_or_buffer='../output/extended.csv',
                        sep=",",
                        skiprows=1,
                        header=None,
                        dtype={0: int, 1: int, 2: int},
                        engine='python')
matrix_df.columns = ["a","b","data"]
Reader.df_col_normalize(matrix_df,"data",{-1:-0.5,0:0.5})
matrix_df.head()

,a,b,data
0,0,0,-0.5
1,0,1,-0.5
2,0,2,-0.5
3,0,3,-0.5
4,0,4,-0.5


In [2]:
# 1 HOT ENCODER
import Utils.Reader as Reader
Reader.oneHotEncoder(name="Type", colsToDelete=["Data"],colstoOneHot=["Type"],dfPath='../data/data_ICM_length.csv')

In [2]:
# DF SPLITTER
import Utils.Reader as Reader
Reader.df_splitter("Rewatch",[0,10,20],dtype={0: int, 1: int},name="rewatches_total_",dfPath='../data/rewatches/rewatches_total.csv',colstoDefine=["UserID","Rewatch"])

In [1]:
# DF PREPROCESSING MODE
# 1-> Displayed (counts times a given Item has been displayed(in impressions list) to the user
# 2-> Extended (adds an interaction (-1) if a User has an item in impression list
# 3-> Count rewatches of each User-Item pair
# 4-> Count all rewatches of each user
import Utils.Reader as Reader
Reader.read_train_csr(matrix_path="../data/interactions_and_impressions.csv",preprocess=5,saving=True)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Preprocessing with mode: 5


Passing through all dataset to gather infos...: 100%|██████████| 5826506/5826506 [06:49<00:00, 14230.75it/s] 


5826506


<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [1]:
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample_double

stackedURM,stackedICM=Reader.stacker(ICM=Reader.read_ICM_rewatches( matrix_format="..."))

URM1=Reader.read_train_csr()
URM_train, URM_test,URM_train2, URM_test2 = split_train_in_two_percentage_global_sample_double(URM1,stackedURM ,train_percentage=0.80)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 41629
Unique FeatureID in the URM are 24507
Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506


In [2]:
URM_train, URM_test,URM_train2, URM_test2

(<41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 841138 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 210690 stored elements in Compressed Sparse Row format>,
 <66136x41629 sparse matrix of type '<class 'numpy.float64'>'
 	with 841863 stored elements in Compressed Sparse Row format>,
 <66136x41629 sparse matrix of type '<class 'numpy.float64'>'
 	with 209965 stored elements in Compressed Sparse Row format>)

In [6]:
841138+210690

1051828

In [7]:
841863+209965

1051828

In [8]:
URM1,stackedURM

(<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1554640 stored elements in Compressed Sparse Row format>,
 <66136x41629 sparse matrix of type '<class 'numpy.float64'>'
 	with 2606468 stored elements in Compressed Sparse Row format>)

In [9]:
# BEST MODEL UNTIL NOW
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
import Utils.Reader as Reader


evaluator_validation=EvaluatorHoldout(URM_test,cutoff_list=[10])
recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

EvaluatorHoldout: Ignoring 1613 ( 3.9%) Users that have less than 1 test interactions
ITEMCFCBF_ITEMKNNCF: URM Detected 24507 (37.1%) users with no interactions.
ITEMCFCBF_ITEMKNNCF: URM Detected 17122 (41.1%) items with no interactions.
ItemKNNCFRecommender: URM Detected 24507 (37.1%) users with no interactions.
ItemKNNCFRecommender: URM Detected 17122 (41.1%) items with no interactions.
UserKNNCFRecommender: URM Detected 24507 (37.1%) users with no interactions.
UserKNNCFRecommender: URM Detected 17122 (41.1%) items with no interactions.
Similarity column 41629 (100.0%), 3837.55 column/sec. Elapsed time 10.85 sec
Similarity column 66136 (100.0%), 2085.70 column/sec. Elapsed time 31.71 sec
[[-0.19513494  2.9882257   5.9388323  ... -0.22396901 -0.22396901
  -0.22396901]
 [-0.22396901  1.4909124   0.62245244 ... -0.22396901 -0.22396901
  -0.22396901]
 [ 1.2615072  10.265362    9.373283   ... -0.22396901 -0.22396901
  -0.22396901]
 ...
 [ 0.10706653  2.195683    1.3285962  ... -0.2239690

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.000662,0.00116,0.001092,0.000161,0.000274,0.001574,0.000574,0.000824,0.006497,0.001592,...,0.961253,0.006246,0.961253,0.048662,9.543942,0.993852,0.110509,0.723943,5.238155,0.654463


In [10]:
URM1

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [9]:
URM_csr

<41629x27968 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [10]:
stacked_URM

<41634x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 1577731 stored elements in Compressed Sparse Row format>

In [2]:
# SUBMITTING PREDICTION
import Utils.Writer as Reader
from Utils.Writer import NameRecommender

submit= Reader.Writer(NameRecommender=NameRecommender.ItemKNNCBF, URM=URM_csr,stackedICM=stacked_ICM)
submit.makeSubmission()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


ItemKNNCBFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 2737.97 column/sec. Elapsed time 10.21 sec
[    0     1     2 ... 41626 41627 41628]


In [3]:
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_csr, train_percentage=0.80)

evaluator_validation=EvaluatorHoldout(URM_test,cutoff_list=[10])
recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

EvaluatorHoldout: Ignoring 1566 ( 3.8%) Users that have less than 1 test interactions
ITEMCFCBF_ITEMKNNCF: URM Detected 3461 (12.4%) items with no interactions.
ItemKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 2404.29 column/sec. Elapsed time 11.63 sec
Similarity column 41629 (100.0%), 1528.68 column/sec. Elapsed time 27.23 sec
[[-0.22624192  2.041109    4.0071144  ... -0.26737946 -0.26737946
  -0.26737946]
 [-0.21830659  1.201427    0.46382955 ... -0.26737946 -0.26737946
  -0.26737946]
 [ 1.169401    8.307266    7.973392   ... -0.26737946 -0.26737946
  -0.26737946]
 ...
 [ 0.40044022  4.1257215   2.8658717  ... -0.26737946 -0.26737946
  -0.26737946]
 [-0.1516939  -0.26737946 -0.26737946 ... -0.26737946 -0.26737946
  -0.26737946]
 [ 0.23713225  4.414356    2.9892635  ... -0.26737946 -0.26737946
  -0.26737946]]
[[-2.63508465e-02  1.22362261e+01  2.2742

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


[[-0.24383812  5.07447     1.9010568  ... -0.33043572 -0.33043572
  -0.33043572]
 [ 0.09230953  2.8509474   3.051913   ... -0.33043572 -0.33043572
  -0.33043572]
 [-0.16443294  2.2503252   1.529164   ... -0.33043572 -0.33043572
  -0.33043572]
 ...
 [-0.33043572 -0.33043572 -0.33043572 ... -0.33043572 -0.33043572
  -0.33043572]
 [-0.26073456 -0.33043572 -0.33043572 ... -0.33043572 -0.33043572
  -0.33043572]
 [ 0.24769135  5.524698    4.1466866  ... -0.33043572 -0.33043572
  -0.33043572]]
[[-0.02727508  1.2437285   0.13979527 ... -0.02727508 -0.02727508
  -0.02727508]
 [-0.02727508  2.7666905   1.604662   ... -0.02727508 -0.02727508
  -0.02727508]
 [-0.02727508 -0.02727508  0.01324373 ... -0.02727508 -0.02727508
  -0.02727508]
 ...
 [-0.02727508  0.18281312  0.05813963 ... -0.02727508 -0.02727508
  -0.02727508]
 [-0.02727508 -0.02727508 -0.02727508 ... -0.02727508 -0.02727508
  -0.02727508]
 [-0.02727508  2.2489882   2.5651     ... -0.02727508 -0.02727508
  -0.02727508]]
[[-0.35671768 -0

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.050131,0.08706,0.081783,0.025321,0.042926,0.164792,NaN,0.06216,0.324065,0.202437,...,0.962382,0.311874,0.962382,0.043274,9.562483,0.993929,0.112211,0.725369,3.271727,0.412391


In [13]:
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_csr, train_percentage=0.80)

evaluator_validation=EvaluatorHoldout(URM_test,cutoff_list=[10])
recommender_ItemKNNCBF = ItemKNNCBFRecommender(URM_train, stacked_ICM)
recommender_ItemKNNCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCBF)
result_df


EvaluatorHoldout: Ignoring 1575 ( 3.8%) Users that have less than 1 test interactions
ItemKNNCBFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 1846.35 column/sec. Elapsed time 15.15 sec


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40054 (100.0%) in 43.33 sec. Users per second: 924


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.039729,0.064625,0.059243,0.021401,0.033479,0.123676,NaN,0.047562,0.242123,0.158085,...,0.962166,0.232963,0.962166,0.082747,11.219829,0.99896,0.21451,0.851011,1.258706,0.486782


In [ ]:
recommender_ItemKNNCF = IALSRecommender(URM_train)
recommender_ItemKNNCF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCF)
result_df